# [RNA2json](https://www.rna.gov.it/RegistroNazionaleTrasparenza/faces/pages/OpenDataAiuti.jspx)

In [16]:
import xml.etree.ElementTree as ET
import pandas as pd

import json
import xmltodict

import os
# os.chdir('c:/users/paolo/Downloads')
os.chdir('c:/users/pvolterr/Downloads')

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [4]:
YM = '2023_12'
periodo = 'OpenData_Aiuti_'+YM
filename =periodo+'.xml'

In [5]:
%%time

with open(filename, encoding='utf-8') as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

json_data = json.dumps(data_dict)

with open(periodo+".json", "w") as outfile:
    outfile.write(json_data)

# print(json.dumps(data_dict, indent=4))

CPU times: total: 26.8 s
Wall time: 28.1 s


In [10]:
## verifica
tree = ET.parse(filename)
root = tree.getroot()

# 

## convertire il json
<br>

In [3]:
jsonfile = periodo+'.json'

with open(jsonfile, 'r',encoding='utf-8') as myfile:
    data=myfile.read()

obj = json.loads(data)
obj.update(obj.pop("LISTA_AIUTI"))
del obj['@xmlns']

## salvare il json pulito

In [9]:
with open(YM+".json", "w") as outfile: 
    json.dump(obj, outfile)

## convertirlo in dataframe
<br>

```
df.groupby('CAR')['CODICE_FISCALE_BENEFICIARIO'].count()

# oppure
df = pd.DataFrame(obj)
#df = df.drop(['@xmlns'], axis=1)
df.info()

```

In [8]:
lst = (obj['AIUTO'])
df = pd.DataFrame(lst)

## json to parquet

In [121]:
df['COMPONENTI_AIUTO'] = df['COMPONENTI_AIUTO'].astype(str)
#df['COMPONENTI_AIUTO'] = df['COMPONENTI_AIUTO'].astype(str)
table = pa.Table.from_pandas(df) # Convert DataFrame to Arrow Table
# Write Arrow Table to Parquet file
pq.write_table(table, YM+'.parquet')

## [json to duckdb](https://duckdb.org/docs/guides/import/json_import.html) direttamente

- create table prov as select * FROM read_json_auto('c:/users/paolo/downloads/2016_11.json'); 
- INSERT INTO prov SELECT * FROM read_json_auto('c:/users/paolo/downloads/2023_11.json');

In [37]:
import duckdb
con = duckdb.connect() # connect to an in-memory database

In [38]:
# create the table "my_table" from the DataFrame "my_df"
con.execute("CREATE TABLE my_table AS SELECT * FROM df")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# insert into the table "my_table" from the DataFrame "my_df"
con.execute("INSERT INTO my_table SELECT * FROM df")

In [1]:
import pyarrow.parquet as pq
import pandas as pd
import glob
import duckdb
import os

# os.chdir('D:/duckdb/files/RNA2') 
os.chdir('c:/users/paolo/downloads')
con = duckdb.connect()
con.execute("PRAGMA threads=2") # enable automatic query parallelization
con.execute("PRAGMA enable_object_cache") # enable caching of parquet metadata

In [11]:
%%time
print(con.execute("SELECT COUNT(distinct(CF)) CF FROM '2023_12.parquet'").df())

       CF
0  144153
CPU times: total: 31.2 ms
Wall time: 27.4 ms


In [12]:
df = con.execute("SELECT *  FROM '*.parquet'").df()

In [13]:
df.shape


(4862867, 11)

In [15]:
df.head(20)

,CAR,concedente,MISURA,CF,impresa,regione,COR,concessione,ID,AIUTO,NOMINALE
0,16781,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...,FCS - Accordi SNSI e Calcolo ad alte prestazioni,00059390815,"""BONO & DITTA SOCIETA' PER AZIONI"" O IN FORMA ...",Sicilia,10149552,2023-01-11,11893455,536058.53,536058.53
1,16781,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...,FCS - Accordi SNSI e Calcolo ad alte prestazioni,00059390815,"""BONO & DITTA SOCIETA' PER AZIONI"" O IN FORMA ...",Sicilia,10149552,2023-01-11,11893455,19023.95,192861.50
2,16781,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...,FCS - Accordi SNSI e Calcolo ad alte prestazioni,00059390815,"""BONO & DITTA SOCIETA' PER AZIONI"" O IN FORMA ...",Sicilia,10149552,2023-01-11,11893456,553679.63,553679.63
3,16781,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...,FCS - Accordi SNSI e Calcolo ad alte prestazioni,00059390815,"""BONO & DITTA SOCIETA' PER AZIONI"" O IN FORMA ...",Sicilia,10149552,2023-01-11,11893456,35707.90,362000.41
4,16781,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...,FCS - Accordi SNSI e Calcolo ad alte prestazioni,03286320829,"ELMI SOCIETA' A RESPONSABILITA' LIMITATA E, IN...",Sicilia,10149553,2023-01-11,11893457,323041.55,323041.55
5,16781,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...,FCS - Accordi SNSI e Calcolo ad alte prestazioni,03286320829,"ELMI SOCIETA' A RESPONSABILITA' LIMITATA E, IN...",Sicilia,10149553,2023-01-11,11893457,11464.28,116222.90
6,16781,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...,FCS - Accordi SNSI e Calcolo ad alte prestazioni,03286320829,"ELMI SOCIETA' A RESPONSABILITA' LIMITATA E, IN...",Sicilia,10149553,2023-01-11,11893458,122855.55,122855.55
7,16781,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...,FCS - Accordi SNSI e Calcolo ad alte prestazioni,03286320829,"ELMI SOCIETA' A RESPONSABILITA' LIMITATA E, IN...",Sicilia,10149553,2023-01-11,11893458,7923.20,80324.00
8,16375,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...,Sostegno del Fondo per la crescita sostenibile...,00186830287,ANTONIO CARRARO S.P.A.,Veneto,10157221,2023-01-18,11904603,180873.75,180873.75
9,16375,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...,Sostegno del Fondo per la crescita sostenibile...,00186830287,ANTONIO CARRARO S.P.A.,Veneto,10157221,2023-01-18,11904603,161688.04,904368.75


In [8]:
df.sort_values(by=['impresa'], inplace=True)

In [14]:
df.drop_duplicates(subset=['CODICE_FISCALE_BENEFICIARIO'], keep='first', inplace=True)

In [16]:
df.to_parquet('RNA_CF.parquet')

### Backup

In [ ]:
# calcolare le righe di ogni file
# https://stackoverflow.com/questions/66388995/finding-the-number-of-rows-for-all-files-within-a-folder

import glob

# 1. list all text files in the directory
rel_filepaths = glob.glob("c:/users/paolo/downloads/*.xml") 
# 2. (optional) create a function to read the number of rows in a file
def count_rows(filepath):
  res = 0
  f = open(filepath, 'r',encoding='utf-8')
  res = len(f.readlines())
  f.close()

  return res

# 3. iterate over your files and use the count_row function
counts = [count_rows(filepath) for filepath in rel_filepaths]

print(counts)

In [7]:
%%time
# convertire velocemente un json in parquet
YM = '2023_12'
periodo = 'OpenData_Aiuti_'+YM
filename =periodo+'.xml'

jsonfile = periodo+'.json'

CPU times: total: 0 ns
Wall time: 0 ns


In [8]:
with open(jsonfile, 'r',encoding='utf-8') as myfile:
    data=myfile.read()

obj = json.loads(data)
obj.update(obj.pop("LISTA_AIUTI"))
del obj['@xmlns']

with open(YM+".json", "w") as outfile: 
    json.dump(obj, outfile)

In [15]:
lst = (obj['AIUTO'])
df = pd.DataFrame(lst)

df['COMPONENTI_AIUTO'] = df['COMPONENTI_AIUTO'].astype(str)
#df['COMPONENTI_AIUTO'] = df['COMPONENTI_AIUTO'].astype(str)
table = pa.Table.from_pandas(df) # Convert DataFrame to Arrow Table
# Write Arrow Table to Parquet file
pq.write_table(table, YM+'.parquet')
print(YM+' fatto')

2023_12 fatto
